<a href="https://colab.research.google.com/github/hailusong/colab-god-idclass/blob/master/god_idclass_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Export Custom Trained Google Object Detection for Inference

Environment variables setup.<br>
**Tensorflow runtime version list** can be found at [here](https://cloud.google.com/ml-engine/docs/tensorflow/runtime-version-list)

In [0]:
DEFAULT_HOME='/content'
TF_RT_VERSION='1.13'
PYTHON_VERSION='3.5'

YOUR_GCS_BUCKET='id-norm'
YOUR_PROJECT='orbital-purpose-130316'

Select the right model from [this official list](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md):

| model | dataset | datetime | notes |
| - |  - | - | - |
| ssd_inception_v2 | coco | 2018_01_28 | |
| ~~ssd_inception_v3~~ | ~~pets~~ | ~~11_06_2017~~ | |
| ssd_mobilenet_v2 | coco | 2018_03_29 | |
| faster_rcnn_resnet101 | coco | 11_06_2017 | |

In [0]:
MODEL_NAME = 'ssd_mobilenet_v2'
PRETRAINED_DATASET = 'coco'
PRETRAINED_TS = '2018_03_29'
PRETRAINED_MODEL_NAME = f'{MODEL_NAME}_{PRETRAINED_DATASET}_{PRETRAINED_TS}'
PIPELINE_CONFIG_NAME = f'pipeline_{MODEL_NAME}'

In [0]:
# This is for faster_rcnn_resnet101
# CHECKPOINT_NUMBER = 3485

# This is for ssd_mobilenet_v2
CHECKPOINT_NUMBER = 5197


## Session and Environment Verification (Destination - Local)

Establish security session with Google Cloud

In [0]:
from google.colab import auth
auth.authenticate_user()


################# RE-RUN ABOVE CELLS IF NEED TO RESTART RUNTIME #################

Verify Versions: TF, Python, IPython and prompt_toolkit (these two need to have compatible version), and protoc

In [0]:
import tensorflow as tf
print(tf.__version__)
assert(tf.__version__.startswith(TF_RT_VERSION + '.')), f'tf.__version__ {tf.__version__} not matching with specified TF runtime version env variable {TF_RT_VERSION}'

1.13.1


In [0]:
!python -V
!ipython --version
!pip show prompt_toolkit
!protoc --version

Python 3.6.7
5.5.0
Name: prompt-toolkit
Version: 1.0.15
Summary: Library for building powerful interactive command lines in Python
Home-page: https://github.com/jonathanslenders/python-prompt-toolkit
Author: Jonathan Slenders
Author-email: UNKNOWN
License: UNKNOWN
Location: /usr/local/lib/python3.6/dist-packages
Requires: six, wcwidth
Required-by: jupyter-console, ipython
libprotoc 3.0.0


## Install Google Object Detection API in Colab
Reference is https://colab.research.google.com/drive/1kHEQK2uk35xXZ_bzMUgLkoysJIWwznYr


### Downgrade prompt-toolkit to 1.0.15 (Destination - Local)
Run this **ONLY** if the Installation not Working

In [0]:
# !pip install 'prompt-toolkit==1.0.15'

### Google Object Detection API Installation (Destination - Local)

In [0]:
!apt-get install -y -qq protobuf-compiler python-pil python-lxml
![ ! -e {DEFAULT_HOME}/models ] && git clone --depth=1 --quiet https://github.com/tensorflow/models.git {DEFAULT_HOME}/models
!ls {DEFAULT_HOME}/models

AUTHORS     CONTRIBUTING.md    LICENSE	 README.md  samples    WORKSPACE
CODEOWNERS  ISSUE_TEMPLATE.md  official  research   tutorials


In [0]:
import os
os.chdir(f'{DEFAULT_HOME}/models/research')
!pwd

/content/models/research


*From Wikipedia ...*: 

**protocol buffers** are a language-neutral, platform-neutral extensible mechanism for serializing structured data – think XML, but smaller, faster, and simpler. 

You define how you want your data to be structured once, then you can **use special generated source code to easily write and read your structured data to and from a variety of data streams and using a variety of languages**.

Remember **.proto defines structured data** and **protoc generates the source code** the serailize/de-serialize.

In [0]:
!protoc object_detection/protos/*.proto --python_out=.
# !ls object_detection/protos/*.proto
# !cat object_detection/protos/anchor_generator.proto
!ls {DEFAULT_HOME}/models/research/object_detection/builders/anchor*

/content/models/research/object_detection/builders/anchor_generator_builder.py
/content/models/research/object_detection/builders/anchor_generator_builder_test.py


#### Add Google Object Detection API into System Path

In [0]:
import sys
sys.path.append(f'{DEFAULT_HOME}/models/research')
sys.path.append(f'{DEFAULT_HOME}/models/research/slim')

Note that ! calls out to a shell (in a **NEW** process), while % affects the **SAME** process associated with the notebook.

Since we append pathes to sys.path, we **HAVE TO** use % command to run the Python

Also it is **IMPORTANT** to have **%matplotlib inline** otherwise %run model_builder_test.py will **cause function attribute error** when accessing matplotlib.pyplot attributes from **iPython's run_line_magic** 

In [0]:
# !find . -name 'inception*' -print
%matplotlib inline

In [0]:
# If see the error 'function' object has no attribute 'called', just run the %matplotlib cell and this cell AGAIN 
%run object_detection/builders/model_builder_test.py

............s...
----------------------------------------------------------------------
Ran 16 tests in 0.103s

OK (skipped=1)


## Export Custom Trained Model

In [0]:
def del_all_flags(FLAGS, excls):
  flags_dict = FLAGS._flags()
  keys_list = [keys for keys in flags_dict]
  for keys in keys_list:
    if keys in excls:
      print(f'SKIPPING exclusion attribute {keys}')
      continue

    print(f'removing attribute {keys}')
    FLAGS.__delattr__(keys)

# if running inside IPython notebook, the python session will be maintained across
# cells, so does the tf.app.flags. That will cause flags defined twice error
# if we %run the app multiple times. The workaroud is to always clean up
# the flags before defining them.
# --------------------------
# flags = tf.app.flags
# del_all_flags(flags.FLAGS, [])

In [0]:
# !gsutil cp gs://${YOUR_GCS_BUCKET}/model_dir/model.ckpt-${CHECKPOINT_NUMBER}.* .

%run object_detection/export_inference_graph.py \
    --logtostderr \
    --input_type image_tensor \
    --pipeline_config_path gs://{YOUR_GCS_BUCKET}/data/{PIPELINE_CONFIG_NAME}_processed.config \
    --trained_checkpoint_prefix gs://{YOUR_GCS_BUCKET}/model_dir_{MODEL_NAME}/model.ckpt-{CHECKPOINT_NUMBER} \
    --output_directory gs://{YOUR_GCS_BUCKET}/exported_graphs_{MODEL_NAME}


Instructions for updating:
Use tf.cast instead.


W0327 20:19:14.580830 140589220386688 deprecation.py:323] From /content/models/research/object_detection/exporter.py:309: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Colocations handled automatically by placer.


W0327 20:19:14.594055 140589220386688 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/tensor_array_ops.py:162: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use tf.cast instead.


...

I0327 20:21:09.187453 140589220386688 builder_impl.py:449] No assets to write.


INFO:tensorflow:SavedModel written to: gs://id-norm/exported_graphs/saved_model/saved_model.pb


I0327 20:21:19.653808 140589220386688 builder_impl.py:414] SavedModel written to: gs://id-norm/exported_graphs/saved_model/saved_model.pb


INFO:tensorflow:Writing pipeline config file to gs://id-norm/exported_graphs/pipeline.config


I0327 20:21:20.228893 140589220386688 config_util.py:184] Writing pipeline config file to gs://id-norm/exported_graphs/pipeline.config
